In [3]:
"""
Deep Deterministic Policy Gradient (DDPG), Reinforcement Learning.
P2P network, net bit rate, energy harvesting example for training.
Thanks to : https://github.com/MorvanZhou/Reinforcement-learning-with-tensorflow/tree/master/contents/9_Deep_Deterministic_Policy_Gradient_DDPG
Using:
tensorflow 1.0
"""
import math
import tensorflow as tf
import numpy as np
import gym
import time
import EH_P2P
import DDPG_CLASS

np.random.seed(1)
tf.random.set_seed(1)



#####################  hyper parameters  ####################

MAX_EPISODES = 2000
MAX_EP_STEPS = 120
LR_A = 0.0002   # learning rate for actor
LR_C = 0.0002  # learning rate for critic
GAMMA = 0.999    # reward discount
REPLACEMENT = [
    dict(name='soft', tau=0.01),
    dict(name='hard', rep_iter_a=600, rep_iter_c=500)
][0]            # you can try different target replacement strategies
MEMORY_CAPACITY = 40000
BATCH_SIZE = 80

OUTPUT_GRAPH = False
    
env=EH_P2P.EH_P2P()
env.Chanpower()
env.Solarread()
    
state_dim = 3 #channel,battery，solar
action_dim = 1 #Transmission power
action_bound = 1 [0,1]

tip=1
tip2=1
snr=-10

for temp in range(1):
   #for snr in range(-10,10,2):
    for modulation in range(1):
        var = 10

        # Using the functional API style to create placeholders in TF2
        S = tf.keras.Input(shape=(state_dim,), dtype=tf.float32, name='s')
        R = tf.keras.Input(shape=(1,), dtype=tf.float32, name='r')
        S_ = tf.keras.Input(shape=(state_dim,), dtype=tf.float32, name='s_')

        # Assigning to DDPG_CLASS
        DDPG_CLASS.S = S
        DDPG_CLASS.R = R
        DDPG_CLASS.S_ = S_

        # Actor and Critic initialization
        actor = DDPG_CLASS.Actor(state_dim, action_dim, action_bound, LR_A, REPLACEMENT)
        critic = DDPG_CLASS.Critic(state_dim, action_dim, LR_C, GAMMA, REPLACEMENT, actor.a, actor.a_)
        actor.add_grad_to_graph(critic.a_grads)
        M = DDPG_CLASS.Memory(MEMORY_CAPACITY, dims=2 * state_dim + action_dim + 1)

        # Initializing variables
        checkpoint = tf.train.Checkpoint(actor=actor, critic=critic)
        checkpoint_manager = tf.train.CheckpointManager(checkpoint, "folder_for_nn_noise/EH_save_net", max_to_keep=100)

        if OUTPUT_GRAPH:
            tf.summary.create_file_writer("logs/")

        print("modulation=", modulation, "snr=", snr)

        for i in range(MAX_EPISODES):
    
            s = env.reset_P2P(snr)

            ep_reward = 0
            for j in range(MAX_EP_STEPS):
                a = actor.choose_action(s)
                a = np.random.normal(a, var)
                a = np.clip(a, 0, 1)
                s_, r, info = env.step_P2P([a, modulation]) # input modulation 0:qpsk,1:8psk,2:16qam

                M.store_transition(s, a, r , s_)

                if M.pointer > MEMORY_CAPACITY:
                    # tip and tip2 are only for printing
                    if tip == 1:
                        print("memory full", j, i)
                        tip = 0
                    var *= 0.9995  # decay the action randomness
                    if tip2 == 1 and var < 0.00000001:
                        print("var zero", j, i)
                        tip2 = 0
                    
                    b_M = M.sample(BATCH_SIZE)
                    b_s = b_M[:, :state_dim]
                    b_a = b_M[:, state_dim: state_dim + action_dim]
                    b_r = b_M[:, -state_dim - 1: -state_dim]
                    b_s_ = b_M[:, -state_dim:]

                    critic.learn(b_s, b_a, b_r, b_s_)
                    actor.learn(b_s)

                s = s_
                
                ep_reward += r

            if i % 30 == 0:
                print("net bit rate=", r, "action", a, "solar,channel,battery", s, "epoch", i)
                print("ave_reward", ep_reward / (j + 1))
                
        # Saving checkpoint
        save_path = checkpoint_manager.save()
        print("Save to path: ", save_path)

print("----------------------------END--------------------------------")

<>:43: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:43: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
/var/folders/yd/0j058frj781_p8p0d_z_jkxh0000gp/T/ipykernel_5426/594363985.py:43: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  action_bound = 1 [0,1]
/var/folders/yd/0j058frj781_p8p0d_z_jkxh0000gp/T/ipykernel_5426/594363985.py:43: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  action_bound = 1 [0,1]
/var/folders/yd/0j058frj781_p8p0d_z_jkxh0000gp/T/ipykernel_5426/594363985.py:43: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  action_bound = 1 [0,1]
/var/folders/yd/0j058frj781_p8p0d_z_jkxh0000gp/T/ipykernel_5426/594363985.py:43: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  action_bound = 1 [0,1]
/var/folders/yd/0j058frj781_p8p0d_z_jkxh0000gp/T/ipykernel_5426/594363985.py:43: SyntaxWarning

TypeError: 'int' object is not subscriptable

net bit rate= 0.0 action [1.63935587e-10] solar,channel,battery [array([0.608032]) 0.28186701229864086 array([2.583328])] 12590.535717190172
net bit rate= 199998.26253519946 action [1.] solar,channel,battery [array([0.79216]) 0.16644346233101426 array([0.708032])] 15267.788957447448
net bit rate= 0.0 action [2.7429523e-43] solar,channel,battery [array([0.687136]) 0.015391263909440678 array([1.600192])] 15052.74967635664
net bit rate= 0.0 action [1.70205514e-17] solar,channel,battery [array([0.61024]) -0.15933629465604893 array([2.387328])] 14843.683708629464
net bit rate= 198511.91921287775 action [1.] solar,channel,battery [array([0.680232]) -0.34469881757705934 array([0.71024])] 17359.686934715057
net bit rate= 0.0 action [2.55374013e-34] solar,channel,battery [array([0.43556]) -0.5246418837900655 array([1.490472])] 17125.09657073242
net bit rate= 82.93793644838571 action [1.] solar,channel,battery [array([0.426896]) -0.6794080484964884 array([0.53556])] 16897.86778894197
net bit rat